In [11]:
import numpy as np
import pandas as pd

In [12]:
df=pd.read_csv("ml-100k/item_cc_genre.csv")

In [13]:
df['movie_title']=df['movie_title'].apply(lambda x:x.lower().strip())

In [14]:
df['director']=df["director"].astype(str)
df['director']=df['director'].apply(lambda x: x.lower().replace(" ",""))


In [15]:
df['actor1']=df["actor1"].astype(str)
df['actor1']=df['actor1'].apply(lambda x: x.lower().replace(" ",""))
#ensuring Case variasion in the actor's name is not identified as a different actor 

In [16]:
df['actor2']=df["actor2"].astype(str)
df['actor2']=df['actor2'].apply(lambda x: x.lower().replace(" ",""))

In [17]:
df['director'] = df['director'].apply(lambda x: x+" "+x+" "+x )
#adding weight to director by replicating the direcor name thrice

In [18]:
df['year']=df['year'].astype(str)

In [19]:
df['hybrid'] = df['director'] +" "+ df['actor1'] +" "+ df['actor2'] +" "+ df['genre']+" "+df['year']

In [20]:
df['hybrid']=df['hybrid'].astype(str)

In [21]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

In [22]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')

In [23]:
count_matrix = count.fit_transform(df['hybrid'])

In [24]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [25]:
titles = df['movie_title']
indices = pd.Series(df.index, index=df['movie_title'])


In [72]:
def get_recommendations(title):
    try:
        title=title.lower().strip()
        idx = indices[title]
#         print(idx)
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:31]
        movie_indices = [i[0] for i in sim_scores]
        d=df[df["movie_title"]==title]["director"].to_string().split()[1]
        a1=df[df["movie_title"]==title]["actor1"].to_string().split()[1]
        a2=df[df["movie_title"]==title]["actor2"].to_string().split()[1]
       
        return titles.iloc[movie_indices].head(15),d,a1,a2
    
    except:
        return 0
    

In [73]:
q=get_recommendations("toy story")


In [74]:
q


(1217                                       goofy movie, a
 420                       aladdin and the king of thieves
 994                                        big green, the
 94                                                aladdin
 818                                             space jam
 991                                              hercules
 843                                    that thing you do!
 62                                      santa clause, the
 1064                                                balto
 1410    land before time iii: the time of the great gi...
 7                                                    babe
 540                                            pocahontas
 1468                                     gumby: the movie
 558                         kid in king arthur's court, a
 25                                 brothers mcmullen, the
 Name: movie_title, dtype: object, 'johnlasseter', 'tomhanks', 'timallen')